In [2]:
import z3
from lib.ts import Ts
from lib.prove import prove_inductive

In [3]:
# The smart contract as a transition system.
ts = Ts('erc20')

owner = 100
sender = z3.BitVec('sender', 16)
receiver = z3.BitVec('receiver', 16)
amount = z3.Int('amount')
p = z3.BitVec('p',16)
q = z3.BitVec('q',16)
address = z3.BitVec('address',16)


totalSupply, totalSupplyOut = ts.add_var(z3.IntSort(), name='totalSupply')

# Balances as an array, mapping from address to Integer.
balances, balancesOut = ts.add_var(z3.ArraySort(z3.BitVecSort(16), z3.IntSort()), name='balance')

# Allowance 
allowance, allowanceOut = ts.add_var(z3.ArraySort(z3.BitVecSort(16), 
        z3.ArraySort(z3.BitVecSort(16), z3.IntSort())),
        name='allowance')

# Track the sum of all balances.
sumOfBalance, sumOfBalanceOut = ts.add_var(z3.IntSort(), name='sumOfBalance')

In [4]:
ts.Init = z3.And(totalSupply == 0,
    z3.ForAll(p, balances[p]==0),
    z3.ForAll([p,q], allowance[p][q] == 0),
    sumOfBalance == 0
    )

In [5]:
def updateAllowance(_a, _sender, _operator, _amount):
    return z3.Update(_a, _sender, z3.Update(_a[_sender],_operator, _amount))

In [6]:
tr_mint = z3.And(sender==owner, amount > 0, 
                address > 0, 
               totalSupplyOut == totalSupply + amount,
               balancesOut == z3.Update(balances,address,balances[address]+amount),
               allowanceOut == allowance,
               sumOfBalanceOut == sumOfBalance+amount)

tr_burn = z3.And(sender==owner, amount > 0, amount <= balances[p], 
                totalSupplyOut == totalSupply - amount,
                balancesOut == z3.Update(balances,p,balances[p]-amount),
                allowanceOut == allowance,
                sumOfBalanceOut == sumOfBalance-amount)

tr_transfer = z3.And(balances[sender] >= amount, amount > 0, 
        balancesOut == z3.Update(z3.Update(balances,sender,balances[sender]-amount), 
            receiver, balances[receiver]+amount),
        totalSupplyOut == totalSupply,
        sumOfBalanceOut == sumOfBalance,
        allowanceOut == allowance)

operator = z3.BitVec('receiver', 16)
tr_transfer_from = z3.And(balances[sender] >= amount, amount > 0, 
        allowance[sender][operator] >= amount,
        balancesOut == z3.Update(z3.Update(balances,sender,balances[sender]-amount), 
            receiver, balances[receiver]+amount),
        allowanceOut == updateAllowance(allowance, sender, operator, allowance[sender][operator]-amount),
        sumOfBalanceOut == sumOfBalance,
        totalSupplyOut == totalSupply
    )

tr_approve = z3.And(amount > 0, 
    allowanceOut == updateAllowance(allowance, sender, operator, amount),
    balancesOut == balances,
    totalSupplyOut == totalSupply,
    sumOfBalanceOut == sumOfBalance
    )

ts.Tr = z3.Or(
    tr_mint, 
    tr_burn, tr_transfer, 
    tr_transfer_from, 
    tr_approve
    )

In [7]:
# Property as an inductive invariant. 
# ts_property = z3.ForAll(p, balances[p] >= 0)

# The sum of all account balance equals to total supply.
ts_property = sumOfBalance == totalSupply

In [8]:
prove_inductive(ts, ts_property)

Prove init => property.
proved
Prove property is inductive.
proved
